# Compiling and Benchmarking the BEiT model

### In this notebook, we will run through the compilation and benchmarking code for the BEiT model. 

##### The first step is to compile the model for inferentia. Here are the necessary imports.

In [ ]:
from transformers import BeitFeatureExtractor, BeitForImageClassification
import torch
from datasets import load_dataset
import neuronperf.torch
import torch_neuron
import os

#### Then we load the model from Hugging Face. https://huggingface.co/docs/transformers/model_doc/beit

In [ ]:
feature_extractor = BeitFeatureExtractor.from_pretrained("microsoft/beit-base-patch16-224")
model = BeitForImageClassification.from_pretrained("microsoft/beit-base-patch16-224")
model = model.eval()

These experiments were initially ran on an Inf.2x large instance. As such, the benchmarking was compiled with pipeline sizes of 1 and 4, with batch sizes of 1 through 10. The training loop is simple, compiling the models to the directory that this notebook exists in. Changing the file path will affect the benchmarking script below. This part of the script can be run in the notebook, but a lot of efficiency is lost. It is reccommended to run the beitCompile.py file from the compile folder locally on your instance rather than in the notebook.

In [ ]:
pipeline_sizes = [1, 4]
batch_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for ncp in pipeline_sizes:
    for bs in batch_sizes:
        model_file = f"beit_neuron_ncp{ncp}_bs{bs}.pt"
        inputs = torch.randn(bs, 3, 224, 224)
        print(f"ncp: {ncp}  bs: {bs}")

        if not os.path.exists(model_file):
            print("Attempting model compilation")
            nmod = torch.neuron.trace(model, example_inputs=inputs, compiler_args=['--neuroncore-pipeline-cores', f"{ncp}"], strict=False)
            nmod.save(model_file)
            del(nmod) # we need to release the model from memory so it doesn't affect benchmarking later on
        else:
            print(f"Found previously compiled model. Skipping compilation\n")

#### The models have been compiled in the folder that this notebook exists in. Now we can commence the benchmarking. We iterate throguh the pipline sizes and the batch sizes, creating separate csv files for each size. For more information on the details of the neuronperf library, please see the documentation here. https://awsdocs-neuron.readthedocs-hosted.com/en/latest/neuron-guide/neuronperf/index.html

In [ ]:
for ncp in pipeline_sizes:
    for bs in batch_sizes:
        model_file = f"beit_neuron_ncp{ncp}_bs{bs}.pt"
        report_file = f"beit_neuron_ncp{ncp}_bs{bs}_benchmark.csv"
        inputs = torch.randn(bs, 3, 224, 224)
        print(f"ncp: {ncp}  bs: {bs}")

        if not os.path.exists(report_file):
            reports = neuronperf.torch.benchmark(model_filename=model_file, inputs=inputs, batch_sizes=[bs], pipeline_sizes=[ncp])
            neuronperf.print_reports(reports)
            neuronperf.write_csv(reports, report_file)
        else:
            print(f"Report file {report_file} already exists. Skipping this benchmark run.\n")

### You can aggregate the beit data frames from the compilation here, and look at your results in an organized fashion.

In [ ]:
import pandas as pd
from glob import glob

"""
For sorting through the created dataframes
"""

dataframes = []

for csv in glob("beit*.csv"):
    dataframes.append(pd.read_csv(csv))
    print(csv)

aggr_df = pd.concat(dataframes)
aggr_df

# Lowest p90 latency
lowest_cost = aggr_df.sort_values(by='latency_p90', ascending=True)[0:5]

# Cheapest Price
lowest_price = aggr_df.sort_values(by='cost per 1 m instances', ascending=True)[0:5]

# Highest average throughput
highest_throughput = aggr_df.sort_values(by='throughput average', ascending=False)[0:5]

To save the dataframes. You can specify the filepath further if you wish to store these files in a directory other than the current one.

In [ ]:
lowest_cost.to_csv('layout_lowest_latency.csv')
lowest_price.to_csv('layout_lowest_latency.csv')
highest_throughput.to_csv('layout_lowest_latency.csv')
